In [ ]:
import datetime
from typing import Any

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain.tools import tool
from pydantic import BaseModel, Field
from tavily import TavilyClient

from chain_reaction.config import APIKeys, ModelBehavior, ModelName

# Define a tool

- A LangChain tool is a wrapper that makes a function callable by a LLM agent. 
- It provides the LLM with a description of what the function does, what parameters it expects, and handles the execution when the LLM decides to use it.

## How Tools Work

When you define a tool, you're creating a structured interface that includes:

1. A name and description (so the LLM knows when to use it)
    - The tool will be referenced by the function's name
    - The function's docstring will serve as the description for the tool.
2. Parameter schemas (so the LLM knows what inputs to provide)
3. The actual function logic to execute


## Best Practices

- **Write clear, specific descriptions**. The LLM relies entirely on your description to decide when to use the tool. Be explicit about what it does and when it should be used. For example, "Search for current weather data for a given city" is better than "Get weather."
- **Keep tools focused and single-purpose**. Each tool should do one thing well rather than trying to handle multiple distinct operations. If you need to search products and get product details, create two separate tools rather than one complex tool with modes.
- **Define explicit parameter schemas**. Use Pydantic models or clear type hints to specify exactly what parameters your tool accepts. Include descriptions for each parameter to help the LLM understand what values to provide.
- **Handle errors gracefully**. Tools should catch exceptions and return meaningful error messages that the LLM can understand and potentially recover from, rather than crashing the entire agent.
- **Return structured data when possible**.** Instead of returning raw strings, consider returning dictionaries or Pydantic models that the LLM can more easily parse and reason about.

In [ ]:
@tool
def calculate_square_root(number: float) -> float:
    """Calculates the square root of a given number.

    Args:
        number (float): The number to calculate the square root of.

    Returns:
        float: The square root of the number.
    """
    return number**0.5


@tool
def calculate_square(number: float) -> float:
    """Calculates the square of a given number.

    Args:
        number (float): The number to calculate the square of.

    Returns:
        float: The square of the number.
    """
    return number**2

# Invoke the tool

In [ ]:
# Calling the tool with raise an error
calculate_square_root(16)

In [ ]:
# Instead, use the invoke method
calculate_square_root.invoke({"number": 16})

# Create an Agent w/ Tools

In [ ]:
# Load API keys from .env file
api_keys = APIKeys()

# Initialize a chat model with your API key
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.factual().model_dump(),
)


# Create a response model
class CalculationResult(BaseModel):
    """Response model for calculation results."""

    result: float = Field(description="The result of the calculation.")


# Initialize an agent using the chat model & tools
agent = create_agent(
    model=chat_model,
    tools=[calculate_square_root, calculate_square],
    system_prompt="""
    You're a helpful assistant that can perform mathematical calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)

In [ ]:
response = agent.invoke(input={"messages": [HumanMessage(content="What's square root of 32,451?")]})
calc_result: CalculationResult = response["structured_response"]

In [ ]:
response_2 = agent.invoke(input={"messages": [HumanMessage(content=f"What's square of {calc_result.result}?")]})
response_2["structured_response"]

# Create web search agent

In [ ]:
# Initialize Tavily client
tavily_client = TavilyClient(api_key=api_keys.tavily.get_secret_value())

In [ ]:
# Define a tool for searching the web using Tavily
@tool
def search_web(query: str) -> dict[str, Any]:
    """Performs a web search using Tavily.

    Args:
        query (str): The search query.

    Returns:
        dict[str, Any]: The search results.
    """
    return tavily_client.search(query=query)


search_web.invoke({"query": "When will it snow next in Centennial, CO?"})

In [ ]:
# Create a response model
class WeatherResult(BaseModel):
    """Response model for weather results."""

    forecast_date: datetime.date = Field(description="The date of the weather forecast.")
    chance_of_snow: float = Field(description="The chance of snow in the specified location.", ge=0, le=100)
    amount_of_snow: float = Field(description="The expected amount of snow in inches.", ge=0)
    temperature: float = Field(description="The expected temperature in Fahrenheit.")
    snow_start: int | None = Field(
        description="The hour when snow is expected to start (24-hour format). None if no snow is expected.",
        ge=0,
        le=23,
    )
    snow_end: int | None = Field(
        description="The hour when snow is expected to end (24-hour format). None if no snow is expected.", ge=0, le=23
    )


# Initialize an agent using the chat model & tools
agent = create_agent(
    model=init_chat_model(
        model=ModelName.CLAUDE_HAIKU,
        timeout=None,
        max_retries=2,
        api_key=api_keys.anthropic,
        **ModelBehavior.factual().model_dump(),
    ),
    tools=[search_web],
    system_prompt="""
    You're a helpful assistant that can search the web for weather information.
    Use the provided web search tools to answer user questions accurately.
    """,
    response_format=WeatherResult,
)

response = agent.invoke(
    input={"messages": [HumanMessage(content="Will centennial, CO have a white Christmas in 2025?")]}
)
response["structured_response"]

In [ ]:
response = agent.invoke(input={"messages": [HumanMessage(content="When will it snow next in Centennial, CO?")]})
response["structured_response"]

In [ ]:
response